In [62]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict
import simpler_sf
simpler_sf.simple_salesforce()
import simple_salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest

In [63]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [64]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [65]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
#nombreArchivoHost = "VentaCARE_ASSISTANCE_" + '22112023' + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2024-02-21
21022024
VentaCARE_ASSISTANCE_21022024.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_21022024.xlsx


In [66]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [67]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [68]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [69]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape


c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(14061, 17)

In [70]:
archivoBCI.dtypes


POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [71]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,28149644,1,FEMENINO,HIJA,LEONOR ISADORA URRA ARANCIBIA,2023-05-15,TIARE.ARANCIBIA.VERGARA@GMAIL.,NO INFORMADO,0
1,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,MASCULINO,HIJO,MAXIMILIANO ALONSO LPEZ ARANCIBIA,2016-04-16,TIARE.ARANCIBIA.VERGARA@GMAIL.,NO INFORMADO,0
2,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,FEMENINO,TITULAR MASCULINO,YENIFER VALERIA ROJAS ROMAN,1990-06-19,yenifer.rojas.r@gmail.com,SANTIAGO,954068927
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,FEMENINO,TITULAR MASCULINO,VALESKA BELEN SILVA BUSTOS,1989-08-04,sb.valeska@gmail.com,SANTIAGO,953810699
4,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,MASCULINO,HIJO,SANTIAGO JESS SALAZAR SILVA,2022-02-02,NO INFORMADO,NO INFORMADO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14006,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,12442680,4,FEMENINO,TITULAR MASCULINO,LINA ESTRELLA ESCUDERO ALCAZAR,1973-06-17,lina.escudero@medelycia.cl,PADRE PEDHUARD 4345,950195701
14007,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,8683459,6,MASCULINO,ESPOSO,MEDEL ECHEVERRIA MARCO ANTONIO,1966-09-16,marco.medel@duratray.com,SANTIAGO,0
14008,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,20416974,8,MASCULINO,HIJO,MEDEL ESCUDERO MANUEL .,2000-11-15,,SN,0
14009,2645511,2023-09-04,2024-09-03,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,17773545,0,17773545,0,FEMENINO,TITULAR FEMENINO,CORDERO TORREJON CAROLINA ANDREA,1991-04-11,c.andrea.c.t@gmail.com,SANTIAGO,988242375


In [72]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [73]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5224\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [74]:
archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,28149644,1,...,,166479770,281496441,2611266,Mujer,,,,Leonor Isadora,Urra Arancibia
1,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,...,,166479770,253536888,2611266,Hombre,,,,Maximiliano Alonso,Lpez Arancibia
2,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,...,954068927,175896805,175896805,2611247,Mujer,yenifer.rojas.r@gmail.com,Santiago,954068927,Yenifer Valeria,Rojas Roman
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,...,953810699,173089821,173089821,2611256,Mujer,sb.valeska@gmail.com,Santiago,953810699,Valeska Belen,Silva Bustos
4,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,...,,173089821,277123916,2611256,Hombre,,,,Santiago Jess,Salazar Silva
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14006,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,12442680,4,...,950195701,124426804,124426804,2642887,Mujer,lina.escudero@medelycia.cl,Padre Pedhuard 4345,950195701,Lina Estrella,Escudero Alcazar
14007,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,8683459,6,...,,124426804,86834596,2642887,Hombre,marco.medel@duratray.com,Santiago,,Medel Echeverria,Marco Antonio
14008,2642887,2023-09-01,2024-08-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,12442680,4,20416974,8,...,,124426804,204169748,2642887,Hombre,,,,Medel Escudero,Manuel .
14009,2645511,2023-09-04,2024-09-03,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,17773545,0,17773545,0,...,988242375,177735450,177735450,2645511,Mujer,c.andrea.c.t@gmail.com,Santiago,988242375,Cordero Torrejon,Carolina Andrea


In [75]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [76]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """


' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [77]:
""" results = sf.query_all(
    
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales'
  
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["sfRut",]]


pacienteProgr.shape """


' results = sf.query_all(\n    \n    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = \'BCI Seguros Individuales\'\n  \n)\npacienteProgr = pd.DataFrame(results["records"])\n# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc\n# String acctName = carol.Account.Name;\n\npacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))\n\npacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()\n\npacienteProgr = pacienteProgr[["sfRut",]]\n\n\npacienteProgr.shape '

In [78]:
#pacienteProgr.head()

In [79]:

campaña_BCI = "BCI Seguros Individuales"


query_pacientePrograma = (
    "select Id,Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña_BCI +"'"
)
PAcientes_q_ya_estan = sf.smart_query(query_pacientePrograma)

PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]
PAcientes_q_ya_estan.head()



100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


,Id,Paciente__r.IdentificationId__pc,Paciente__r.FirstName,Paciente__r.LastName,N_Documento_Titular__c,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c,sfRut
0,a40RN0000011clpYAA,177675407,Sebastián Jesus,Ramos Díaz,177675407,2584550,76712269-1,BCI Seguros,True,Titular,177675407
1,a40RN0000012pm0YAA,137314401,Loreto Alejandra,Hofer Duran,137314401,2584550,76712269-1,BCI Seguros,True,Titular,137314401
2,a40RN0000012swbYAA,162007858,Maria Gabriela,Gonzalez Miranda,162007858,2584551,76712269-1,BCI Seguros,True,Titular,162007858
3,a40RN0000013pOnYAI,162891227,José,Orrego Ramírez,162891227,2584551,76712269-1,BCI Seguros,True,Titular,162891227
4,a40RN0000017FEfYAM,18233040k,Paulo,Vera Muñoz,18233040k,2584551,76712269-1,BCI Seguros,True,Titular,18233040k


In [80]:
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].astype(str).str.upper()
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace(" ", "")
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(pacienteProgr["sfRut"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]


archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI.drop_duplicates(["Rut Carga", "Rut Titular"], keep="last")

archivoBCI = archivoBCI[
    ["Rut Carga", "sexo", "direccion", "email", "celular", "nombres", "apellidos"]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
archivoBCI["Cuenta_validada__c"] = "True"
""" archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Seguros Individuales/derty.xlsx", index=False
) """


archivoBCI.head()


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c
7958,95150845,Hombre,Santiago,w.reval@hotmail.com,956784027,Walter Mirko,Valdivia Helo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
8092,260694014,Hombre,,rogerv990@gmail.com,930938831,Roger Enrique,Marcano Marcano,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
9230,202398774,Hombre,.,claudiamunozc@yahoo.es,,Benjamin Claudio,Leiva Munoz,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
9231,174362165,Hombre,Santiago,salgadosergio891@gmail.com,964830780,Salgado Salgado,Salgado Sergio,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
9232,256145332,Mujer,.,,,Emily Fernanda,Salgado Isidro,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False


In [81]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """
""" soql_query =  select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales' 
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
#df = df.drop(columns="Id")
list_of_records = df.to_dict("index") """

""" derty = sf.quick_search("153298939")
print(derty) """

""" for listado in list_of_records:
    list_of_records[listado]["Paciente__r"]["IdentificationId__pc"]
    juanka =  len(list_of_records)
    #print(list_of_records[listado]["Paciente__r"]["IdentificationId__pc"])
    print(list_of_records[listado]["Id"]) """
    
list_of_records[1]

{'Id': 'a1Y8c00000CzGHTEA3',
 'Paciente__r': OrderedDict([('attributes',
               OrderedDict([('type', 'Account'),
                            ('url',
                             '/services/data/v42.0/sobjects/Account/0018c00002mVtpiAAC')])),
              ('IdentificationId__pc', '154512675')])}

Weeeeeeeeeeeenooo


In [82]:
# rut = "124942012"
def traerId(rut):
    if (len(rut)>=8):
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""        

In [84]:

archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(lambda y: traerId(y))
archivoBCI["Programas_por_Empresas__c"] = "a1W8c000008TCWxEAO"
archivoBCI.head()

001RN000006hgDVYAY 95150845
001RN000006hitaYAA 260694014
001RN000006hhHcYAI 202398774
001RN000006hk4FYAQ 174362165
001RN000006hmCQYAY 256145332
001RN000006hjGEYAY 267905185
001RN000006hhCoYAI 140151963
001RN000006hmSYYAY 253438886
001RN000006hfChYAI 224999003
001RN000006hk5pYAA 102833678
001RN000006hmM6YAI 231609156
0018c00002aySmEAAU 189262620
0018c00002ayUDZAA2 267780889
0018c00002ayS8bAAE 252448810
001RN000006hebbYAA 97913838
001RN000006hkQkYAI 185692116
001RN000006hhZTYAY 226944745
001RN000006hgbiYAA 184503360
001RN000006hkDtYAI 159678741
001RN000006hoAzYAI 18039298K
001RN000006hgbjYAA 282673479
001RN000006hoCbYAI 155574828
001RN000006hk4GYAQ 105918798
001RN000006hW9NYAU 171655730
001RN000006hezjYAA 281650181
001RN000006hoEDYAY 92386112
001RN000006hin9YAA 177554758
001RN000006hoB0YAI 7050605K
001RN000006hgwdYAA 199617273
001RN000006hoHRYAY 161786551
001RN000006hiX3YAI 198393282
001RN000006hc1vYAA 158048108
001RN000006hlEkYAI 200991028
001RN000006hkKIYAY 120906542
001RN000006hoB1YAI

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
7958,95150845,Hombre,Santiago,w.reval@hotmail.com,956784027,Walter Mirko,Valdivia Helo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hgDVYAY,a1W8c000008TCWxEAO
8092,260694014,Hombre,,rogerv990@gmail.com,930938831,Roger Enrique,Marcano Marcano,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hitaYAA,a1W8c000008TCWxEAO
9230,202398774,Hombre,.,claudiamunozc@yahoo.es,,Benjamin Claudio,Leiva Munoz,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hhHcYAI,a1W8c000008TCWxEAO
9231,174362165,Hombre,Santiago,salgadosergio891@gmail.com,964830780,Salgado Salgado,Salgado Sergio,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hk4FYAQ,a1W8c000008TCWxEAO
9232,256145332,Mujer,.,,,Emily Fernanda,Salgado Isidro,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,001RN000006hmCQYAY,a1W8c000008TCWxEAO


In [86]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False)

archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
7958,95150845,Hombre,Santiago,w.reval@hotmail.com,956784027,Walter Mirko,Valdivia Helo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hgDVYAY,a1W8c000008TCWxEAO
8092,260694014,Hombre,,rogerv990@gmail.com,930938831,Roger Enrique,Marcano Marcano,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hitaYAA,a1W8c000008TCWxEAO
9230,202398774,Hombre,.,claudiamunozc@yahoo.es,,Benjamin Claudio,Leiva Munoz,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hhHcYAI,a1W8c000008TCWxEAO
9231,174362165,Hombre,Santiago,salgadosergio891@gmail.com,964830780,Salgado Salgado,Salgado Sergio,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN000006hk4FYAQ,a1W8c000008TCWxEAO
9232,256145332,Mujer,.,,,Emily Fernanda,Salgado Isidro,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,001RN000006hmCQYAY,a1W8c000008TCWxEAO


In [ ]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [ ]:

Subir_pacientes = archivoBCI
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 003RN00000JcOhYYAV', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
001RN000006hgDVYAY 95150845
Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 003RN00000JcSpxYAF', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
vacio
Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 003RN00000JcMflYAF', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
001RN000006hhHcYAI

' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [ ]:
prueba = "153298939"
sf.query(
    format_soql(
        "SELECT Id,lastname FROM Account WHERE IdentificationId__pc = {last_name}",
        last_name=prueba,
    )
)

OrderedDict([('totalSize', 1),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])),
                            ('Id', '0018c00002anTmeAAE'),
                            ('LastName', 'Martinez Matus')])])])

In [ ]:
""" sf.Account.create(
    {
        "IdentificationId__pc": "12345678910",
        "HealthCloudGA__Gender__pc": "Mujer",
        "FirstName": "johan",
        "LastName": "Smith",
        "PersonEmail": "example@example.com",
        "BillingCountry": "Chile",
        "Identification_Type__c": "RUT",
        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",
        "RecordTypeId": "0128c000002Byp4AAC",
    }
)
 """

# sf.Account.update(1, {'Name': 'Test Account', ContactId: contact_id}) # doctest: +SKIP

' sf.Account.create(\n    {\n        "IdentificationId__pc": "12345678910",\n        "HealthCloudGA__Gender__pc": "Mujer",\n        "FirstName": "johan",\n        "LastName": "Smith",\n        "PersonEmail": "example@example.com",\n        "BillingCountry": "Chile",\n        "Identification_Type__c": "RUT",\n        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",\n        "RecordTypeId": "0128c000002Byp4AAC",\n    }\n)\n '

In [ ]:
""" import pandas as pd
sf.bulk.Account.query("SELECT Id, Email FROM Account")

df = pd.DataFrame.from_dict(data,orient='columns').drop('attributes',axis=1) """

import pandas as pd

df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id, Phone FROM Account WHERE Id IN ({})", dataframe_to_sfdc_list(df,column)))


NameError: name 'column' is not defined

In [ ]:
contact = sf.Account.get('0018c00002amLcCAAU')

In [ ]:
# sf = simple_salesforce.Salesforce(...)

query = "SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Account.IdentificationId__pc, Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c,Empresa_Principal__c,Country from ServiceAppointment"

df = sf.smart_query(query)
df

In [ ]:
df.head(-50)

In [ ]:
rut = "153298939"

""" busquedaDatos = (
    "FIND {" + rut + "} "
) """
derty = sf.quick_search(rut)
derty


In [ ]:
df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
archivoBCI["IdentificationId__pc"]
df.head()

In [ ]:
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id FROM Account WHERE IdentificationId__pc IN ({})", dataframe_to_sfdc_list(pacienteProgr,"sfRut")))

In [ ]:
pacienteProgr.head()

Codigo que dio github Copilot para la solucion del error de 0 en envio de correos

In [ ]:
import win32com.client as win32
import os
import shutil

nombre_reporte = "Vida Security - Ferronor"
nombre_carpeta = "F:/Reportes SF/Vida Security/"
nombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

# create Excel application object
excel = win32.gencache.EnsureDispatch('Excel.Application')

# set ScreenUpdating property to False to hide Excel application
excel.ScreenUpdating = False

# open workbook
workbook = excel.Workbooks.Open(nombre_archivo_final)

# do some work with the workbook

# save and close workbook
workbook.Save()
workbook.Close()

# set ScreenUpdating property to True to show Excel application
excel.ScreenUpdating = True

# quit Excel application
excel.Quit()

# release COM object
del excel

envio_de_email(
    destinatarios=["marialaura.perez@security.cl",],
    concopia=[
        "dnunez@careassistance.com",
        "dgarcia@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx",
    activo=1,
)

In [ ]:
import win32com.client as win32
import os
import shutil
import datetime as dt

def crear_carpeta(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)

def actualizar(Archivo_origen):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(Archivo_origen)
    wb.RefreshAll()
    wb.Save()
    wb.Close()

def envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ";".join(destinatarios)
    mail.CC = ";".join(concopia)
    mail.Subject = asunto
    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"
    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)
    mail.Send()

nombre_reporte = "Zurich - AMSA"
nombre_carpeta = "F:/Reportes SF/Zurich/"
nombre_archivo_final = nombre_carpeta + str(dt.date.today()) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dt.date.today())
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

envio_de_email(
    destinatarios=[
        "clara.angelica.munoz@zurich.com",
        "claudia.ivonne.zapata@zurich.com",
        "maria.moreno.poblete@zurich.com",
        "nadia.pino@zurich.com",
    ],
    concopia=[
        "dgarcia@careassistance.com",
        "dnunez@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx"